# 1. 소개
* 조명 - 치즈케이크
* 프로젝트명 - 
* 분석목적 - 최근 유행하는 치즈케이크 집의 분포를 알아보자

# 2. 조원소개와 역할 (사진포함)
* 김다영
* 김민기
* 서준영
* 이수진

# 3. 데이터 출처와 전처리전의 데이터 구조(테이블구조로 샘플수와 속성 설명 포함)
## (1) 데이터 수집 방법 : 크롤링
* 데이터 출처 : 인스타그램
인스타그램에서 #치즈케이크 해시태그로 검색하여 최근 게시물 데이터를 확보했다.

목표 샘플수는 10000개를 목표로 했으나, 인스타그램의 검색량 제한으로 중간에 멈췄다.

따라서 크롤링 과정을 2단계로 나누어 네 명이서 크롤링을 분담하였다.

특징: 위치 정보를 입력하지 않는 글은 저장되지 않는다

### 사전 환경 설정

In [13]:
from selenium import webdriver
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import re

### RAW 데이터 수집

In [30]:
#사전정보 입력
handle = '' #인스타그램 아이디
pwd = ''  #인스타그램 비밀번호

keyword= '치즈케이크'   #검색할 키워드
num_of_pages = 800 #몇 번 스크롤 할 건지; e.g. 200 --> 1730개 게시글 긁어짐
driver_dir = 'c:/Temp/chromedriver.exe'    #크롬드라이버 경로
k = 1 # k는 1~4로 각각 맡음

#1단계 : 검색 페이지에서 글 URL 가져오기
def InstagramUrlFromKeyword (browser,keyword,num_of_pagedowns):
    keyword_url_encode=quote(keyword)
    url='https://www.instagram.com/explore/tags/'+keyword+'/?hl=ko'
    browser.get(url)
    time.sleep(3)
    login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
    login.click()
    time.sleep(3)
    user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
    pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
    user_id.send_keys(handle)
    pw.send_keys(pwd)
    button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
    button.click()
    time.sleep(3)
    arr_href=[]
    body=browser.find_element_by_tag_name('body')
    for i in range(num_of_pagedowns):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        post=browser.find_elements_by_class_name('v1Nh3')
        for j in post:
            href_str=j.find_element_by_css_selector('a').get_attribute('href')
            arr_href.append(href_str)
    return set(arr_href)    

char = r'#[\d\w]+'
char2 = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!#?]')

#2단계 : URL 정보를 4명분으로 나누기
arr = list(arr)
text1 = arr[:threshold]
text2 = arr[threshold: 2*threshold]
text3 = arr[2*threshold: 3*threshold]
text4 = arr[3*threshold:]
texts = [text1, text2, text3, text4]

for k in range(len(texts)):
    with open('url_part' + str(k+1) + '.txt', 'w') as f:
        for url in texts[k]:
            f.write(url + '\n')


#3단계 : 글 URL에서 필요한 데이터 수집
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    location = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').text
    loc_url = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').get_attribute('href')
    return datetime, user_name, like_count, tags, location, url, loc_url

#4단계 : 크롤링 시행
browser = webdriver.Chrome(driver_dir)
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags', 'Location','URL', 'Location_URL'})

with open('url_part' + str(k) + '.txt', 'r') as f:
    urls = f.readlines()
for url in urls:
    try:
        datetime, user_name, like_count, tags, location, url, loc_url = IdHashTagFromInstagram(browser, url.strip('\n'))
        insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags, 'Location' : location, 'URL':url,'Location_URL' : loc_url}, ignore_index = True)
    except:
        continue

### 지도 URL에서 Latitude, Longitude 수집 크롤링

In [ ]:
# 로그인
browser= webdriver.Chrome(driver_dir)
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle) #아이디
pw.send_keys(pwd) #비밀번호
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)

#주소URL에서 위도, 경도 가져오기
def grab_and_clean_selenium(url):
    browser.get(url)
    try:
        lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
        long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
    except:
        lat = 'N/A'
        long= 'N/A'
    return lat,long

coord = pd.DataFrame(columns=['Latitude','Longitude','Location_URL'])
for url in insta_df['Location_URL']:
    lat, long = grab_and_clean_selenium(url)
    coord = coord.append({'Latitude':lat,'Longitude':long,'Location_URL':url}, ignore_index=True)

insta_df_final = pd.merge(insta_df,coord)
insta_df_final.to_csv('rawdata_final.csv',mode='w', encoding='UTF-8')

#### 크롤링 완료 후 데이터가 잘 수집되었는지 확인

In [20]:
len(insta_df)

1634

## (2) 전처리 전의 데이터 구조
* 샘플수 1633개
* 수집 항목 : 작성날짜, 해시태그, 좋아요 수, 위치, 사용자 아이디, 지도url



In [44]:
insta_df[:10]

,Datetime,Tags,Like_Count,Location,User_Name,URL,Location_URL
0,2019년 12월 22일,"['#부산', '#서울', '#친구', '#밥', '#스타벅스', '#케이크', '...",25,"Busan, South Korea",_ye._.yomi_,https://www.instagram.com/p/B6W9LyMhTjB/,https://www.instagram.com/explore/locations/28...
1,2019년 12월 29일,"['#카페', '#카페그램', '#카페투어', '#카페추천', '#신상카페', '#...",56,향미공간:香味空間,flavor_space,https://www.instagram.com/p/B6pC7XtJ-s7/,https://www.instagram.com/explore/locations/28...
2,2019년 12월 22일,"['#익스큐즈미', '#익스큐즈미카페', '#카페투어', '#카페', '#카페스타그...",33,마곡지구,borami_ing,https://www.instagram.com/p/B6V3bf5gQyF/,https://www.instagram.com/explore/locations/34...
3,2019년 12월 19일,"['#아인슈페너', '#연유카페라떼', '#치즈케이크', '#우리동네카페', '#까...",13,"Seoul, Korea",revi_ewfood,https://www.instagram.com/p/B6PRZImh242/,https://www.instagram.com/explore/locations/21...
4,2019년 12월 29일,"['#디저트', '#치즈케이크', '#바닐라', '#뜨아', '#더벤티']",7,The venti 면목점,kyong_jin_life_record,https://www.instagram.com/p/B6n1h6Fpnrx/,https://www.instagram.com/explore/locations/11...
5,2019년 12월 23일,"['#홍대', '#홍대카페', '#먹스타그램', '#디저트그램', '#치즈케이크']",21,C27 cheesecake&coffee,88.10.21,https://www.instagram.com/p/B6aoUK4BpQV/,https://www.instagram.com/explore/locations/29...
6,2019년 12월 29일,"['#shibuya', '#치즈케이크', '#일본']",87,CCC Cheese Cheers Cafe Shibuya,non1101.___,https://www.instagram.com/p/B6p6mZLnxUg/,https://www.instagram.com/explore/locations/10...
7,2019년 12월 29일,['#어나더룸'],96,Another Room,snjlee_,https://www.instagram.com/p/B6pxwzonI0p/,https://www.instagram.com/explore/locations/87...
8,2019년 12월 29일,[],58,"Goodgoodweather,굿굿웨더",goodgoodweather,https://www.instagram.com/p/B6pEirIJhJS/,https://www.instagram.com/explore/locations/39...
9,2019년 12월 27일,"['#스템스커피', '#stemscoffee', '#듁스커피', '#dukescof...",43,Stemscoffee,stemscoffee,https://www.instagram.com/p/B6j2lCfhZNC/,https://www.instagram.com/explore/locations/18...


#### 크롤링 결과 수집된 데이터를 csv로 저장
euc-kr로 저장하면 인코딩 문제로 오류나서 utf-8로 저장함

In [22]:
insta_df.to_csv('rawdata.csv',mode='w', encoding='euc-kr')

UnicodeEncodeError: 'euc_kr' codec can't encode character '\xe8' in position 46: illegal multibyte sequence

In [23]:
insta_df.to_csv('rawdata.csv',mode='w', encoding='UTF-8')

# 4. 분석방법, 흐름소개
* 분석방법 : 크롤링, 빈도분석, 지도 API를 이용하여 위치정보 텍스트를 위도, 경도로 변환
* 흐름 소개 (동영상, 각종코드 넣기)

## 데이터 탐색

### 시연을 위해 크롤링으로 얻은 raw data 로드

In [48]:
insta_df = pd.read_csv('rawdata.csv', index_col=0, encoding='UTF-8')
insta_df[:10]

,Datetime,Tags,Like_Count,Location,User_Name,URL,Location_URL
0,2019년 12월 22일,"['#부산', '#서울', '#친구', '#밥', '#스타벅스', '#케이크', '...",25,"Busan, South Korea",_ye._.yomi_,https://www.instagram.com/p/B6W9LyMhTjB/,https://www.instagram.com/explore/locations/28...
1,2019년 12월 29일,"['#카페', '#카페그램', '#카페투어', '#카페추천', '#신상카페', '#...",56,향미공간:香味空間,flavor_space,https://www.instagram.com/p/B6pC7XtJ-s7/,https://www.instagram.com/explore/locations/28...
2,2019년 12월 22일,"['#익스큐즈미', '#익스큐즈미카페', '#카페투어', '#카페', '#카페스타그...",33,마곡지구,borami_ing,https://www.instagram.com/p/B6V3bf5gQyF/,https://www.instagram.com/explore/locations/34...
3,2019년 12월 19일,"['#아인슈페너', '#연유카페라떼', '#치즈케이크', '#우리동네카페', '#까...",13,"Seoul, Korea",revi_ewfood,https://www.instagram.com/p/B6PRZImh242/,https://www.instagram.com/explore/locations/21...
4,2019년 12월 29일,"['#디저트', '#치즈케이크', '#바닐라', '#뜨아', '#더벤티']",7,The venti 면목점,kyong_jin_life_record,https://www.instagram.com/p/B6n1h6Fpnrx/,https://www.instagram.com/explore/locations/11...
5,2019년 12월 23일,"['#홍대', '#홍대카페', '#먹스타그램', '#디저트그램', '#치즈케이크']",21,C27 cheesecake&coffee,88.10.21,https://www.instagram.com/p/B6aoUK4BpQV/,https://www.instagram.com/explore/locations/29...
6,2019년 12월 29일,"['#shibuya', '#치즈케이크', '#일본']",87,CCC Cheese Cheers Cafe Shibuya,non1101.___,https://www.instagram.com/p/B6p6mZLnxUg/,https://www.instagram.com/explore/locations/10...
7,2019년 12월 29일,['#어나더룸'],96,Another Room,snjlee_,https://www.instagram.com/p/B6pxwzonI0p/,https://www.instagram.com/explore/locations/87...
8,2019년 12월 29일,[],58,"Goodgoodweather,굿굿웨더",goodgoodweather,https://www.instagram.com/p/B6pEirIJhJS/,https://www.instagram.com/explore/locations/39...
9,2019년 12월 27일,"['#스템스커피', '#stemscoffee', '#듁스커피', '#dukescof...",43,Stemscoffee,stemscoffee,https://www.instagram.com/p/B6j2lCfhZNC/,https://www.instagram.com/explore/locations/18...


### 빈도분석
수집한 데이터가 모두 범주형 데이터이다.

해시태그에 대해 빈도분석을 진행하였다

In [68]:
from collections import Counter

insta_tags = []
for tags in insta_df['Tags']:
    tags = tags.strip('[').strip(']').replace("'", "")
    tag_list = tags.split(', ')
    while '' in tag_list:
        tag_list.remove('')
    insta_tags.extend(tag_list)
    
c = Counter(insta_tags)
    
c.most_common(50)

[('#치즈케이크', 871),
 ('#카페', 172),
 ('#일상', 137),
 ('#디저트', 120),
 ('#카페투어', 107),
 ('#크리스마스', 105),
 ('#cafe', 94),
 ('#카페스타그램', 84),
 ('#cheesecake', 84),
 ('#케이크', 83),
 ('#디저트카페', 80),
 ('#커피', 77),
 ('#coffee', 77),
 ('#마카롱', 76),
 ('#먹스타그램', 75),
 ('#데일리', 75),
 ('#좋아요', 63),
 ('#daily', 54),
 ('#딸기케이크', 51),
 ('#dessert', 49),
 ('#맛집', 49),
 ('#맛스타그램', 49),
 ('#아메리카노', 48),
 ('#스콘', 46),
 ('#딸기', 46),
 ('#맞팔', 46),
 ('#디저트맛집', 46),
 ('#부산카페', 44),
 ('#크리스마스케이크', 43),
 ('#소통', 42),
 ('#티라미수', 40),
 ('#마카롱맛집', 39),
 ('#cake', 39),
 ('#예쁜카페', 37),
 ('#브라우니', 34),
 ('#메리크리스마스', 33),
 ('#먹방', 33),
 ('#팔로우', 31),
 ('#초코케이크', 31),
 ('#마들렌', 30),
 ('#제리치즈케이크', 29),
 ('#선물', 29),
 ('#macarons', 29),
 ('#좋반', 29),
 ('#밀크티', 28),
 ('#커피맛집', 28),
 ('#크리스마스이브', 28),
 ('#세종시마카롱', 28),
 ('#새롬동카페', 28),
 ('#신상카페', 27)]

### (2) 지도 API를 이용하여 위치정보 텍스트를 위도, 경도로 변환